In [1]:
from sqlalchemy import MetaData
metadata = MetaData()

from sqlalchemy import Table,Column, Integer, String
user_table = Table(
    "user_account",
    metadata,
    Column('id',Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname',String)
)

In [2]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [3]:
user_table.c.keys()

['id', 'name', 'fullname']

In [4]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [5]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata,
    Column('id',Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'),nullable=False),
    Column('email_address',String,nullable=False)
)

In [8]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///user.sqlite",echo=True,future=True)
metadata.create_all(engine)

2021-04-27 08:40:40,039 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 08:40:40,040 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2021-04-27 08:40:40,040 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-27 08:40:40,042 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2021-04-27 08:40:40,042 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-27 08:40:40,043 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address",back_populates="user")
    
    def __repr__(self):
        return f"User(id={self.id!r},name={self.name!r}, fullname={self.fullname!r})"


In [10]:
class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))
    user = relationship("User",back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [11]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [12]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///user1.sqlite",echo=True,future=True)
Base.metadata.create_all(engine)

2021-04-27 08:55:53,925 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 08:55:53,926 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2021-04-27 08:55:53,926 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-27 08:55:53,927 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2021-04-27 08:55:53,928 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-27 08:55:53,929 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2021-04-27 08:55:53,929 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-27 08:55:53,930 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2021-04-27 08:55:53,931 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-04-27 08:55:53,932 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2021-04-27 08:55:53,933 INFO sqlalchemy.engine.Engine [no key 0.00065s] ()
2021-04-27 08:55:53,935 INFO sqlalchemy.engine.Engine 
C

In [13]:
squidward = User(name="國堂", fullname="徐國堂")
krabs = User(name='ehkrabs', fullname="Eugene H. Krabss")
squidward

User(id=None,name='國堂', fullname='徐國堂')

In [16]:
from sqlalchemy.orm import Session
session = Session(engine)

In [17]:
session.add(squidward)
session.add(krabs)

In [18]:
session.new

IdentitySet([User(id=None,name='國堂', fullname='徐國堂'), User(id=None,name='ehkrabs', fullname='Eugene H. Krabss')])

In [19]:
session.flush()

2021-04-27 09:14:33,653 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 09:14:33,655 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-04-27 09:14:33,656 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ('國堂', '徐國堂')
2021-04-27 09:14:33,657 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-04-27 09:14:33,658 INFO sqlalchemy.engine.Engine [cached since 0.002793s ago] ('ehkrabs', 'Eugene H. Krabss')


In [20]:
squidward.id

1

In [21]:
session.commit()

2021-04-27 09:17:53,371 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
from sqlalchemy import select
stmt = select(User).where(User.name == '國堂')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row.__class__)
        print(row)

2021-04-27 09:31:47,785 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 09:31:47,786 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-04-27 09:31:47,787 INFO sqlalchemy.engine.Engine [cached since 127.2s ago] ('國堂',)
<class 'sqlalchemy.engine.row.Row'>
(User(id=1,name='國堂', fullname='徐國堂'),)
2021-04-27 09:31:47,789 INFO sqlalchemy.engine.Engine ROLLBACK


In [23]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [26]:
row=session.execute(select(User)).first()

2021-04-27 09:32:59,085 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2021-04-27 09:32:59,086 INFO sqlalchemy.engine.Engine [cached since 24.39s ago] ()


In [27]:
row

(User(id=1,name='國堂', fullname='徐國堂'),)

In [28]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [29]:
row = session.execute(select(User.name, User.fullname)).first()

2021-04-27 09:34:34,267 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2021-04-27 09:34:34,268 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()


In [30]:
user1 = User(name='sandy', fullname='Sandy Cheeks')
user2 = User(name='patrick', fullname='Patrick Star')
with Session(engine) as session:
    session.add(user1)
    session.add(user2)
    session.commit()

2021-04-27 09:43:13,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 09:43:13,658 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-04-27 09:43:13,658 INFO sqlalchemy.engine.Engine [cached since 1720s ago] ('sandy', 'Sandy Cheeks')
2021-04-27 09:43:13,660 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-04-27 09:43:13,660 INFO sqlalchemy.engine.Engine [cached since 1720s ago] ('patrick', 'Patrick Star')
2021-04-27 09:43:13,661 INFO sqlalchemy.engine.Engine COMMIT


In [36]:
with Session(engine) as session:
    sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()
    sandy.name
    sandy.fullname = "Sandy Squirrel" 
    session.commit()

2021-04-27 09:53:46,793 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 09:53:46,795 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-04-27 09:53:46,795 INFO sqlalchemy.engine.Engine [cached since 1446s ago] ('sandy',)
2021-04-27 09:53:46,798 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2021-04-27 09:53:46,799 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('Sandy Squirrel', 3)
2021-04-27 09:53:46,800 INFO sqlalchemy.engine.Engine COMMIT


In [38]:
sandy_fullname = session.execute(select(User.fullname).where(User.id==3)).scalar_one()
sandy_fullname

2021-04-27 09:57:02,779 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2021-04-27 09:57:02,780 INFO sqlalchemy.engine.Engine [cached since 33.1s ago] (3,)


'Sandy Squirrel'

In [56]:
from sqlalchemy import update
with Session(engine) as session:
    session.execute(
        update(User).
        where(User.name=="sandy").
        values(fullname="Sandy Squirrel Extraodinaire")
    )
  

2021-04-27 10:14:43,601 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 10:14:43,602 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2021-04-27 10:14:43,603 INFO sqlalchemy.engine.Engine [cached since 770.3s ago] ('Sandy Squirrel Extraodinaire', 'sandy')
2021-04-27 10:14:43,604 INFO sqlalchemy.engine.Engine ROLLBACK


In [57]:
with Session(engine) as session:
    sandy=session.execute(select(User).where(User.id==3)).scalar_one()
    print(sandy)

2021-04-27 10:14:49,660 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-04-27 10:14:49,661 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2021-04-27 10:14:49,662 INFO sqlalchemy.engine.Engine [cached since 17.7s ago] (3,)
User(id=3,name='sandy', fullname='Sandy Squirrel')
2021-04-27 10:14:49,663 INFO sqlalchemy.engine.Engine ROLLBACK
